## Context-Free Grammars (CFG)

A **Context-Free Grammar (CFG)** is a formal grammar used to describe the syntax of a language.

A CFG is defined as a 4-tuple:

**G = (V, Σ, R, S)**

where:

- **V** → a finite set of *non-terminal symbols* (e.g., `S`, `NP`, `VP`)  
- **Σ** → a finite set of *terminal symbols* (the actual words)  
- **R** → a set of *production rules* of the form  
  `A → α`  
  where `A` is a non-terminal in `V`, and `α` is a sequence of terminals and/or non-terminals  
- **S** → the *start symbol* (`S ∈ V`)  

A CFG generates sentences by starting from the start symbol **S** and repeatedly applying the rules until only terminal symbols remain.



In [4]:
import nltk
from nltk import CFG

# Define a simple CFG
grammar = CFG.fromstring("""
S -> NP VP
NP -> Det N
VP -> V NP
Det -> 'the' | 'a'
N -> 'dog' | 'cat'
V -> 'chases' | 'sees'
""")

# Generate a parser
parser = nltk.EarleyChartParser(grammar)

sentence = ['the', 'dog', 'sees', 'a', 'cat']

# Display all possible parses
for tree in parser.parse(sentence):
    tree.pretty_print()
    tree.draw()


             S              
      _______|____           
     |            VP        
     |        ____|___       
     NP      |        NP    
  ___|___    |     ___|___   
Det      N   V   Det      N 
 |       |   |    |       |  
the     dog sees  a      cat



## 🎲 Probabilistic Context-Free Grammars (PCFG)

A **Probabilistic Context-Free Grammar (PCFG)** extends a CFG by assigning a **probability** to each production rule.

For each non-terminal symbol A, the probabilities of its production rules sum to 1:

`sum over alpha of P(A -> alpha) = 1`

In plain text form:
`P(A -> alpha1) + P(A -> alpha2) + ... = 1`

The probability of a complete parse tree is the product of the probabilities of all rules used in that derivation:

`P(tree) = P(r1) * P(r2) * ... * P(rn)`

(Where each `ri` is a rule used in the derivation.)

Thus, when a sentence admits multiple parse trees, the PCFG picks the parse with the highest `P(tree)`.



In [2]:
from nltk import PCFG, ViterbiParser

# Define a PCFG with probabilities
grammar = PCFG.fromstring("""
S -> NP VP [1.0]
NP -> Det N [0.6] | 'John' [0.4]
VP -> V NP [0.7] | V [0.3]
Det -> 'the' [0.8] | 'a' [0.2]
N -> 'dog' [0.5] | 'telescope' [0.5]
V -> 'sees' [1.0]
""")

parser = ViterbiParser(grammar)

sentence = ['John', 'sees', 'the', 'dog']

# Parse and show the most probable parse tree
for tree in parser.parse(sentence):
    print("Most probable parse with probability:", tree.prob())
    tree.pretty_print()
    tree.draw()


Most probable parse with probability: 0.0672
           S             
  _________|___           
 |             VP        
 |     ________|___       
 |    |            NP    
 |    |         ___|___   
 NP   V       Det      N 
 |    |        |       |  
John sees     the     dog



In [3]:
grammar2 = PCFG.fromstring("""
S -> NP VP [1.0]
NP -> Det N [0.5] | Det N PP [0.5]
VP -> V NP [0.7] | V NP PP [0.3]
PP -> P NP [1.0]
Det -> 'the' [1.0]
N -> 'man' [0.5] | 'telescope' [0.5]
V -> 'saw' [1.0]
P -> 'with' [1.0]
""")

parser = ViterbiParser(grammar2)
sentence = ['the', 'man', 'saw', 'the', 'telescope', 'with', 'the', 'telescope']

for tree in parser.parse(sentence):
    print("Tree probability:", tree.prob())
    tree.pretty_print()


Tree probability: 0.0109375
                 S                                  
      ___________|______                             
     |                  VP                          
     |        __________|______                      
     |       |                 NP                   
     |       |    _____________|____                 
     |       |   |      |           PP              
     |       |   |      |       ____|___             
     NP      |   |      |      |        NP          
  ___|___    |   |      |      |     ___|______      
Det      N   V  Det     N      P   Det         N    
 |       |   |   |      |      |    |          |     
the     man saw the telescope with the     telescope

